## Transportation Problem (TP) with activation costs

In [1]:
import pyomo.environ as pyo
import numpy as np

# Define sets of sources and destinations
N_s = 3
N_d = 4

S = pyo.RangeSet(1, N_s)
D = pyo.RangeSet(1, N_d)

# Define subsets S_d and D_s
D_s = {1: [1, 2, 3],
       2: [1, 3, 4],
       3: [2, 3, 4]}

S_d = {1: [1, 2],
       2: [1, 3],
       3: [1, 2, 3],
       4: [2, 3]}

# Define all parameters needed by the model
S_minus = {1: 10, 2: 10, 3: 10}
S_plus = {1: 50, 2: 80, 3: 60}
D_minus = {1: 50, 2: 10, 3: 70, 4: 10}
D_plus = {1: 50, 2: 40, 3: 70, 4: 50}

LB = 10
UB = 20
C_sd = {(s, d): np.random.randint(LB, UB) for s in S for d in D_s[s]}

# Activation costs
activation_costs = {(s, d): np.random.uniform(5, 15) for s in S for d in D_s[s]}

# Maximum value for x_{s,d}
M = max(S_plus.values())

# Setup model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(S, D, within=pyo.NonNegativeReals)
model.y = pyo.Var(S, D, within=pyo.Binary)

# Constraints
model.C1 = pyo.Constraint(S, rule=lambda model, s: sum(model.x[s, d] for d in D_s[s]) >= S_minus[s])
model.C2 = pyo.Constraint(S, rule=lambda model, s: sum(model.x[s, d] for d in D_s[s]) <= S_plus[s])
model.C3 = pyo.Constraint(D, rule=lambda model, d: sum(model.x[s, d] for s in S_d[d]) >= D_minus[d])
model.C4 = pyo.Constraint(D, rule=lambda model, d: sum(model.x[s, d] for s in S_d[d]) <= D_plus[d])

# New constraint for activation costs
model.C5 = pyo.Constraint(S, D, rule=lambda model, s, d: model.x[s, d] <= M * model.y[s, d])

# Objective
model.obj = pyo.Objective(expr=sum((C_sd[(s, d)] + activation_costs[(s, d)]) * model.x[s, d] for s in S for d in D_s[s]))

# Solve MILP
opt = pyo.SolverFactory('gurobi')
opt.solve(model)

# Display results
for s in S:
    for d in D_s[s]:
        print(f"x[{s},{d}] = {model.x[s, d].value}, y[{s},{d}] = {model.y[s, d].value}")

# Retrieve active routing variables
eps = 0.00001
active_variables = [(f"x[{s},{d}]", model.x[s, d].value) for s in S for d in D_s[s] if model.x[s, d].value >= eps]
print("Active Variables:")
print(active_variables)


x[1,1] = 0.0, y[1,1] = 1.0
x[1,2] = 0.0, y[1,2] = 1.0
x[1,3] = 50.0, y[1,3] = 1.0
x[2,1] = 50.0, y[2,1] = 1.0
x[2,3] = 20.0, y[2,3] = 1.0
x[2,4] = 10.0, y[2,4] = 1.0
x[3,2] = 10.0, y[3,2] = 1.0
x[3,3] = 0.0, y[3,3] = 1.0
x[3,4] = 0.0, y[3,4] = 1.0
Active Variables:
[('x[1,3]', 50.0), ('x[2,1]', 50.0), ('x[2,3]', 20.0), ('x[2,4]', 10.0), ('x[3,2]', 10.0)]
